# Refactored Code Integration and Test Cases

# Part 1: Refactored Code Integration

## Data Selection

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("AppGallery.csv")

In [3]:
df.head()

,Ticket id,Interaction id,Interaction date,Mailbox,Ticket Summary,Interaction content,Innso TYPOLOGY_TICKET,Type 1,Type 2,Type 3,Type 4,Unnamed: 11
0,24310,68365,NaN,support.eu@Service.com,[Company Music] Problems and Suggestions,Mxxxxx@xxxx.com Отправлено с моего телефона Co...,Gallery; Games / Other Requests,Gallery; Games,Others,NaN,NaN,NaN
1,24075,67730,NaN,support.pt@Service.com,[Gallery]Probleme und Vorschläge,Beschreibung Update lässt sich nicht installie...,Gallery; Games / Suggestions &amp; Feedback,Gallery; Games,Problem/Fault,Gallery-Install/Upgrade,Can't update Apps,NaN
2,24075,67762,NaN,support.pt@Service.com,Re: RE : [Gallery]Probleme und Vorschläge Re: ...,Danke Nach Neuinstallation der App funktionier...,Gallery; Games / Suggestions &amp; Feedback,Gallery; Games,Problem/Fault,Gallery-Install/Upgrade,Can't update Apps,NaN
3,23748,66706,NaN,support.pt@Service.com,[Gallery]Problems and suggestions,Description . Contact information Axxxxx@xxxx.com,Gallery; Games / Other Requests,Gallery; Games,Suggestion,Gallery-Use,Others,NaN
4,23740,66684,NaN,support.eu@Service.com,NaN,Ho pagato ma l'applicazione non ha funzionato ...,Gallery; Games / Refund request : app/item pur...,Gallery; Games,Problem/Fault,Third Party APPs,Refund,NaN


In [4]:
# convert the dtype object to unicode string
df['Interaction content'] = df['Interaction content'].values.astype('U')
df['Ticket Summary'] = df['Ticket Summary'].values.astype('U')

In [5]:
#Optional: rename variable names for remebering easily
df["y1"] = df["Type 1"]
df["y2"] = df["Type 2"]
df["y3"] = df["Type 3"]
df["y4"] = df["Type 4"]
df["x"] = df['Interaction content']

df["y"] = df["y2"]

# remove empty y
df = df.loc[(df["y"] != '') & (~df["y"].isna()),]
df.shape

print(df.shape)

(122, 18)


In [6]:
df.head()

,Ticket id,Interaction id,Interaction date,Mailbox,Ticket Summary,Interaction content,Innso TYPOLOGY_TICKET,Type 1,Type 2,Type 3,Type 4,Unnamed: 11,y1,y2,y3,y4,x,y
0,24310,68365,NaN,support.eu@Service.com,[Company Music] Problems and Suggestions,Mxxxxx@xxxx.com Отправлено с моего телефона Co...,Gallery; Games / Other Requests,Gallery; Games,Others,NaN,NaN,NaN,Gallery; Games,Others,NaN,NaN,Mxxxxx@xxxx.com Отправлено с моего телефона Co...,Others
1,24075,67730,NaN,support.pt@Service.com,[Gallery]Probleme und Vorschläge,Beschreibung Update lässt sich nicht installie...,Gallery; Games / Suggestions &amp; Feedback,Gallery; Games,Problem/Fault,Gallery-Install/Upgrade,Can't update Apps,NaN,Gallery; Games,Problem/Fault,Gallery-Install/Upgrade,Can't update Apps,Beschreibung Update lässt sich nicht installie...,Problem/Fault
2,24075,67762,NaN,support.pt@Service.com,Re: RE : [Gallery]Probleme und Vorschläge Re: ...,Danke Nach Neuinstallation der App funktionier...,Gallery; Games / Suggestions &amp; Feedback,Gallery; Games,Problem/Fault,Gallery-Install/Upgrade,Can't update Apps,NaN,Gallery; Games,Problem/Fault,Gallery-Install/Upgrade,Can't update Apps,Danke Nach Neuinstallation der App funktionier...,Problem/Fault
3,23748,66706,NaN,support.pt@Service.com,[Gallery]Problems and suggestions,Description . Contact information Axxxxx@xxxx.com,Gallery; Games / Other Requests,Gallery; Games,Suggestion,Gallery-Use,Others,NaN,Gallery; Games,Suggestion,Gallery-Use,Others,Description . Contact information Axxxxx@xxxx.com,Suggestion
4,23740,66684,NaN,support.eu@Service.com,nan,Ho pagato ma l'applicazione non ha funzionato ...,Gallery; Games / Refund request : app/item pur...,Gallery; Games,Problem/Fault,Third Party APPs,Refund,NaN,Gallery; Games,Problem/Fault,Third Party APPs,Refund,Ho pagato ma l'applicazione non ha funzionato ...,Problem/Fault


## Noise Remover

In [7]:
temp = df
# remove re:
# remove extrac white space
# remove
noise = "(sv\s*:)|(wg\s*:)|(ynt\s*:)|(fw(d)?\s*:)|(r\s*:)|(re\s*:)|(\[|\])|(aspiegel support issue submit)|(null)|(nan)|((bonus place my )?support.pt 自动回复:)"
temp["ts"] = temp["Ticket Summary"].str.lower().replace(noise, " ", regex=True).replace(r'\s+', ' ',
                                                                                        regex=True).str.strip()
temp_debug = temp.loc[:, ["Ticket Summary", "ts", "y"]]

temp["ic"] = temp["Interaction content"].str.lower()
noise_1 = [
    "(from :)|(subject :)|(sent :)|(r\s*:)|(re\s*:)",
    "(january|february|march|april|may|june|july|august|september|october|november|december)",
    "(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)",
    "(monday|tuesday|wednesday|thursday|friday|saturday|sunday)",
    "\d{2}(:|.)\d{2}",
    "(xxxxx@xxxx\.com)|(\*{5}\([a-z]+\))",
    "dear ((customer)|(user))",
    "dear",
    "(hello)|(hallo)|(hi )|(hi there)",
    "good morning",
    "thank you for your patience ((during (our)? investigation)|(and cooperation))?",
    "thank you for contacting us",
    "thank you for your availability",
    "thank you for providing us this information",
    "thank you for contacting",
    "thank you for reaching us (back)?",
    "thank you for patience",
    "thank you for (your)? reply",
    "thank you for (your)? response",
    "thank you for (your)? cooperation",
    "thank you for providing us with more information",
    "thank you very kindly",
    "thank you( very much)?",
    "i would like to follow up on the case you raised on the date",
    "i will do my very best to assist you"
    "in order to give you the best solution",
    "could you please clarify your request with following information:"
    "in this matter",
    "we hope you(( are)|('re)) doing ((fine)|(well))",
    "i would like to follow up on the case you raised on",
    "we apologize for the inconvenience",
    "sent from my huawei (cell )?phone",
    "original message",
    "customer support team",
    "(aspiegel )?se is a company incorporated under the laws of ireland with its headquarters in dublin, ireland.",
    "(aspiegel )?se is the provider of huawei mobile services to huawei and honor device owners in",
    "canada, australia, new zealand and other countries",
    "\d+",
    "[^0-9a-zA-Z]+",
    "(\s|^).(\s|$)"]
for noise in noise_1:
    print(noise)
    temp["ic"] = temp["ic"].replace(noise, " ", regex=True)
temp["ic"] = temp["ic"].replace(r'\s+', ' ', regex=True).str.strip()
temp_debug = temp.loc[:, ["Interaction content", "ic", "y"]]

print(temp.y1.value_counts())
good_y1 = temp.y1.value_counts()[temp.y1.value_counts() > 10].index
temp = temp.loc[temp.y1.isin(good_y1)]
print(temp.shape)

(from :)|(subject :)|(sent :)|(r\s*:)|(re\s*:)
(january|february|march|april|may|june|july|august|september|october|november|december)
(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)
(monday|tuesday|wednesday|thursday|friday|saturday|sunday)
\d{2}(:|.)\d{2}
(xxxxx@xxxx\.com)|(\*{5}\([a-z]+\))
dear ((customer)|(user))
dear
(hello)|(hallo)|(hi )|(hi there)
good morning
thank you for your patience ((during (our)? investigation)|(and cooperation))?
thank you for contacting us
thank you for your availability
thank you for providing us this information
thank you for contacting
thank you for reaching us (back)?
thank you for patience
thank you for (your)? reply
thank you for (your)? response
thank you for (your)? cooperation
thank you for providing us with more information
thank you very kindly
thank you( very much)?
i would like to follow up on the case you raised on the date
i will do my very best to assist youin order to give you the best solution
could you please clarify your request wi

<>:5: SyntaxWarning: invalid escape sequence '\s'
<>:12: SyntaxWarning: invalid escape sequence '\s'
<>:16: SyntaxWarning: invalid escape sequence '\d'
<>:17: SyntaxWarning: invalid escape sequence '\.'
<>:49: SyntaxWarning: invalid escape sequence '\d'
<>:51: SyntaxWarning: invalid escape sequence '\s'
<>:5: SyntaxWarning: invalid escape sequence '\s'
<>:12: SyntaxWarning: invalid escape sequence '\s'
<>:16: SyntaxWarning: invalid escape sequence '\d'
<>:17: SyntaxWarning: invalid escape sequence '\.'
<>:49: SyntaxWarning: invalid escape sequence '\d'
<>:51: SyntaxWarning: invalid escape sequence '\s'
/var/folders/0q/w0rrhgw97mz8mxw_rzvyl2t40000gn/T/ipykernel_82227/1041950880.py:5: SyntaxWarning: invalid escape sequence '\s'
  noise = "(sv\s*:)|(wg\s*:)|(ynt\s*:)|(fw(d)?\s*:)|(r\s*:)|(re\s*:)|(\[|\])|(aspiegel support issue submit)|(null)|(nan)|((bonus place my )?support.pt 自动回复:)"
/var/folders/0q/w0rrhgw97mz8mxw_rzvyl2t40000gn/T/ipykernel_82227/1041950880.py:12: SyntaxWarning: invali

## Translation

In [8]:
import stanza
from stanza.pipeline.core import DownloadMethod
from transformers import pipeline
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer


def trans_to_en(texts):
    t2t_m = "facebook/m2m100_418M"
    t2t_pipe = pipeline(task='text2text-generation', model=t2t_m)

    model = M2M100ForConditionalGeneration.from_pretrained(t2t_m)
    tokenizer = M2M100Tokenizer.from_pretrained(t2t_m)
    nlp_stanza = stanza.Pipeline(lang="multilingual", processors="langid",
                                 download_method=DownloadMethod.REUSE_RESOURCES)

    text_en_l = []
    for text in texts:
        if text == "":
            text_en_l = text_en_l + [text]
            continue

        doc = nlp_stanza(text)
        print(doc.lang)
        if doc.lang == "en":
            text_en_l = text_en_l + [text]
        else:
            lang = doc.lang
            if lang == "fro":  # fro = Old French
                lang = "fr"
            elif lang == "la":  # latin
                lang = "it"
            elif lang == "nn":  # Norwegian (Nynorsk)
                lang = "no"
            elif lang == "kmr":  # Kurmanji
                lang = "tr"

            case = 2

            if case == 1:
                text_en = t2t_pipe(text, forced_bos_token_id=t2t_pipe.tokenizer.get_lang_id(lang='en'))
                text_en = text_en[0]['generated_text']
            elif case == 2:
                tokenizer.src_lang = lang
                encoded_hi = tokenizer(text, return_tensors="pt")
                generated_tokens = model.generate(**encoded_hi, forced_bos_token_id=tokenizer.get_lang_id("en"))
                text_en = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
                text_en = text_en[0]
            else:
                text_en = text

            text_en_l = text_en_l + [text_en]

            print(text)
            print(text_en)

    return text_en_l
	
#Calling translation method
# Note that the we can only translate a limited number of words so we are only translating ticket summary and not interaction content

temp["ts_en"] = trans_to_en(temp["ts"].to_list())

/Users/yixuan/Documents/JupyterLab/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/yixuan/Documents/JupyterLab/venv/lib/python3.12/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
2024-06-12 20:23:13 INFO: Loading these models for language: multilingual ():
| Processor | Package |
-----------------------
| langid    | ud      |

2024-06-12 20:23:13 

en
de
gallery probleme und vorschläge
gallery issues and suggestions
en
en
en
en
en
en
en
pt
gallery problemas e sugestões
gallery problems and suggestions
de
gallery probleme und vorschläge
gallery issues and suggestions
en
pt
gallery problemas e sugestões
gallery problems and suggestions
pt
gallery problemas e sugestões
gallery problems and suggestions
it
gallery problemi e soluzioni
Gallery of Problems and Solutions
ru
gallery проблеми и предложения
Gallery of problems and suggestions
ru
gallery проблеми и предложения
Gallery of problems and suggestions
ru
gallery проблеми и предложения
Gallery of problems and suggestions
no
gallery problemer og forslag
gallery problems and suggestions
no
gallery problemer og forslag
gallery problems and suggestions
ru
gallery проблеми и предложения
Gallery of problems and suggestions
ru
gallery проблеми и предложения
Gallery of problems and suggestions
en
en
en
tr
gallery sorunlar ve öneriler
gallery issues and suggestions
en
sk
gallery problémy a 

In [9]:
temp.head()

,Ticket id,Interaction id,Interaction date,Mailbox,Ticket Summary,Interaction content,Innso TYPOLOGY_TICKET,Type 1,Type 2,Type 3,...,Unnamed: 11,y1,y2,y3,y4,x,y,ts,ic,ts_en
0,24310,68365,NaN,support.eu@Service.com,[Company Music] Problems and Suggestions,Mxxxxx@xxxx.com Отправлено с моего телефона Co...,Gallery; Games / Other Requests,Gallery; Games,Others,NaN,...,NaN,Gallery; Games,Others,NaN,NaN,Mxxxxx@xxxx.com Отправлено с моего телефона Co...,Others,company music problems and suggestions,company,company music problems and suggestions
1,24075,67730,NaN,support.pt@Service.com,[Gallery]Probleme und Vorschläge,Beschreibung Update lässt sich nicht installie...,Gallery; Games / Suggestions &amp; Feedback,Gallery; Games,Problem/Fault,Gallery-Install/Upgrade,...,NaN,Gallery; Games,Problem/Fault,Gallery-Install/Upgrade,Can't update Apps,Beschreibung Update lässt sich nicht installie...,Problem/Fault,gallery probleme und vorschläge,beschreibung update sst sich nicht installiere...,gallery issues and suggestions
2,24075,67762,NaN,support.pt@Service.com,Re: RE : [Gallery]Probleme und Vorschläge Re: ...,Danke Nach Neuinstallation der App funktionier...,Gallery; Games / Suggestions &amp; Feedback,Gallery; Games,Problem/Fault,Gallery-Install/Upgrade,...,NaN,Gallery; Games,Problem/Fault,Gallery-Install/Upgrade,Can't update Apps,Danke Nach Neuinstallation der App funktionier...,Problem/Fault,gallery probleme und vorschläge gallery proble...,danke nach neuinstallation der app funktionier...,gallery probleme und vorschläge gallery proble...
3,23748,66706,NaN,support.pt@Service.com,[Gallery]Problems and suggestions,Description . Contact information Axxxxx@xxxx.com,Gallery; Games / Other Requests,Gallery; Games,Suggestion,Gallery-Use,...,NaN,Gallery; Games,Suggestion,Gallery-Use,Others,Description . Contact information Axxxxx@xxxx.com,Suggestion,gallery problems and suggestions,description contact information,gallery problems and suggestions
4,23740,66684,NaN,support.eu@Service.com,nan,Ho pagato ma l'applicazione non ha funzionato ...,Gallery; Games / Refund request : app/item pur...,Gallery; Games,Problem/Fault,Third Party APPs,...,NaN,Gallery; Games,Problem/Fault,Third Party APPs,Refund,Ho pagato ma l'applicazione non ha funzionato ...,Problem/Fault,,ho pagato ma applicazione non ha funzionato po...,


## Textual data representation

In [10]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

tfidfconverter = TfidfVectorizer(max_features=2000, min_df=4, max_df=0.90)
x1 = tfidfconverter.fit_transform(temp["Interaction content"]).toarray()
x2 = tfidfconverter.fit_transform(temp["ts_en"]).toarray()
X = np.concatenate((x1, x2), axis=1)

## Dealing Data Imbalancing
Some code is duplicated and is commented out for this part 

In [11]:
from sklearn.model_selection import train_test_split
y = temp.y.to_numpy()
# remove bad test cases from test dataset
# Test_size = 0.20
# new_test_size = 0.2
y_series = pd.Series(y)
good_y_value = y_series.value_counts()[y_series.value_counts() >= 3].index
y_good = y[y_series.isin(good_y_value)]
X_good = X[y_series.isin(good_y_value)]
y_bad = y[y_series.isin(good_y_value) == False]
X_bad = X[y_series.isin(good_y_value) == False]
test_size = X.shape[0] * 0.2 / X_good.shape[0]
# print(f"new_test_size: {new_test_size}")
X_train, X_test, y_train, y_test = train_test_split(X_good, y_good,     test_size=test_size, random_state=0)
X_train = np.concatenate((X_train, X_bad), axis=0)
y_train = np.concatenate((y_train, y_bad), axis=0)

## Data Preparation for Modelling
<b> We comment out the following code, because in the "Dealing Data Imbalancing" file, the training and testing data is already defined. </b>

In [12]:
# from sklearn.model_selection import train_test_split
# y = temp.y.to_numpy()
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## Model Selection

In [13]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators=1000, random_state=0)

## Training

In [14]:
classifier.fit(X_train, y_train)

RandomForestClassifier(n_estimators=1000, random_state=0)

## Testing and Display Results

In [15]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

y_pred = classifier.predict(X_test)

p_result = pd.DataFrame(classifier.predict_proba(X_test))
p_result.columns = classifier.classes_
print(p_result)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

    Others  Problem/Fault  Suggestion
0    0.883          0.066       0.051
1    0.018          0.979       0.003
2    0.088          0.785       0.127
3    0.047          0.917       0.036
4    0.164          0.559       0.277
5    0.808          0.113       0.079
6    0.175          0.174       0.651
7    0.205          0.431       0.364
8    0.077          0.909       0.014
9    0.201          0.768       0.031
10   0.083          0.157       0.760
11   0.008          0.981       0.011
12   0.027          0.926       0.047
13   0.450          0.469       0.081
14   0.656          0.205       0.139
15   0.140          0.786       0.074
16   0.016          0.969       0.015
17   0.211          0.710       0.079
18   0.186          0.617       0.197
19   0.192          0.706       0.102
20   0.014          0.982       0.004
21   0.024          0.955       0.021
22   0.067          0.643       0.290
23   0.647          0.222       0.131
24   0.021          0.975       0.004
[[ 1  2  0]


# Part 2: Test Cases

In [ ]:
import unittest
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from locust import HttpUser, task, between
import json
import joblib

Unit testing - Each test will be carried out separately. I will comment it out for other functions when the specific test is carried out.

In [ ]:
class Unit_Test_Data_Model(unittest.TestCase):
    # Test Case 1  - this test case will be performed after preprocessed the data.
    def preprocessing_test(self):
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X_train)  
        assert np.isclose(np.mean(X_scaled), 0, atol=1e-7), "The mean value of X_scaled should be close to 0"
        assert np.isclose(np.std(X_scaled), 1, atol=1e-7), "The standard deviation of X_scaled should be close to 1"   
        # To prevent floating-point precision issues and AssertionError, atol parameter is used.
    
    # Test Case 2 - this test case will be carried out after the model has been trained.
    def model_training_test(self):  
        X, y = make_classification(n_samples=1000, n_features=20, random_state=42)  # Generate synthetic data for testing
        
        RFmodel = RandomForestClassifier(n_estimators=1000, random_state=42)  
        RFmodel.fit(X, y)
    
        assert hasattr(RFmodel, "estimators_"), "The model should have 'estimators_' attribute after training" # Assert that the model has certain attributes after training
        assert len(RFmodel.estimators_) == 1000, "The model should have 100 estimators"   # Validate to ensure that the model behaves as defined

    # Test Case 3 - this test will be performed to validate if the output (multiple classification) of the prediction of the trained model is in the range of possible class labels or not. 
    def model_integration_test(self):  
        X, y = make_classification(n_samples=1000, n_features=20, n_classes=3, random_state=42)
        RFmodel = RandomForestClassifier(n_estimators=1000, random_state=0).fit(X, y)
        predictions = RFmodel.predict(X)
        no_classes = len(np.unique(y))  
        assert set(predictions) <= set(range(no_classes)), "Predictions should be in the range of possible class labels"


if __name__ == '__main__':
    unittest.main()

Load Test (Performance test)

In [ ]:
# Generate a synthetic dataset with make_classification
X_sync, y_sync = make_classification(n_samples=2000, n_features=244, n_classes=3, n_informative=10, random_state=42)

# Get the trained RandomForestClassifier with warm_start
RFmodel = RandomForestClassifier(n_estimators=1000, warm_start=True)                     
RFmodel.fit(X_sync, y_sync)

# Save the model
joblib.dump(RFmodel, 'RFclassifier.joblib')

# After the model is saved as per above, the following part should be in .py file to perform load test of the trained model.
# Load the saved model 
model = joblib.load('RFclassifier.joblib')

class model_users(HttpUser):
    wait_time = between(1, 5)  # Time (between tasks)

    @task
    def predict(self):
        # Generate random data
        sample_data = np.random.rand(1, 244)

        # Convert to JSON payload
        payload = json.dumps(sample_data.tolist())

        # Prediction with synthetic data 
        prediction = model.predict(sample_data)

        # Prediction result log
        print(f"RF Classifier Prediction: {prediction}")

if __name__ == "__main__":
    import os
    os.system("locust -f locustfile.py --host http://localhost")

# end of test case